In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Load data
@st.cache_data
def load_data():
    agent_data = pd.read_excel("Agent.xlsx")
    inventory_data = pd.read_excel("Inventory.xlsx", sheet_name="10304-11434")
    sales_data = pd.read_excel("Sales.xlsx")
    sales_data["SALE DATE"] = pd.to_datetime(sales_data["SALE DATE"])
    return agent_data, inventory_data, sales_data

# Initialize data
agent_data, inventory_data, sales_data = load_data()

# Pre-defined credentials
c_level_users = ["Leticia", "Giselle", "Tuopu", "Sherry", "Heng", "Yi"]
agent_users = agent_data["Full Name"].unique()

# Preset passwords
c_level_password = "123"
agent_password = "456"

# Persistent state for client data
if "client_data" not in st.session_state:
    st.session_state.client_data = {}

if "appointment_requests" not in st.session_state:
    st.session_state.appointment_requests = []

if "purchase_requests" not in st.session_state:
    st.session_state.purchase_requests = []

# Persistent state for login/register flow
if "client_mode" not in st.session_state:
    st.session_state.client_mode = "Login"  # Default mode: Login

# Sidebar for login/register
st.title("🏠 Dream Homes NYC Management App")
st.sidebar.header("Login/Register")

role = st.sidebar.selectbox("Select your role", ["C-Level", "Agent", "Client"])

if role == "Client":
    # Toggle between Register and Login modes
    if st.sidebar.button("Register"):
        st.session_state.client_mode = "Register"
    if st.sidebar.button("Login"):
        st.session_state.client_mode = "Login"

# Username and Password fields
username = st.sidebar.text_input("Username (email for clients)")
password = st.sidebar.text_input("Password", type="password")

# Authentication Logic
if role in ["C-Level", "Agent", "Client"] and (st.session_state.client_mode == "Login" or role != "Client"):
    if st.sidebar.button("Submit"):
        # C-Level Authentication
        if role == "C-Level" and username in c_level_users and password == c_level_password:
            st.success(f"Welcome, {username} (C-Level)")
            show_dashboard = "C-Level"

        # Agent Authentication
        elif role == "Agent" and username in agent_users and password == agent_password:
            st.success(f"Welcome, {username} (Agent)")
            show_dashboard = "Agent"

        # Client Login
        elif role == "Client":
            if username in st.session_state.client_data:
                if password == st.session_state.client_data[username]["password"]:
                    st.success(f"Welcome back, {username} (Client)")
                    show_dashboard = "Client"
                else:
                    st.error("Incorrect password.")
            else:
                st.error("Username not found. Please register first.")

        # Incorrect credentials
        else:
            st.error("Incorrect username or password.")

# Client Registration
if role == "Client" and st.session_state.client_mode == "Register":
    st.sidebar.write("Please register your account.")
    email = st.sidebar.text_input("Enter your email (this will be your username)", key="reg_email")
    new_password = st.sidebar.text_input("Set a new password", type="password", key="reg_new_password")
    confirm_password = st.sidebar.text_input("Confirm password", type="password", key="reg_confirm_password")

    if st.sidebar.button("Register Account"):
        if email and new_password and confirm_password:
            if new_password == confirm_password:
                if email not in st.session_state.client_data:
                    st.session_state.client_data[email] = {"password": new_password}
                    st.success("Registration successful! Please log in.")
                    st.session_state.client_mode = "Login"  # Switch to Login mode after successful registration
                else:
                    st.error("This email is already registered. Please log in.")
            else:
                st.error("Passwords do not match.")

# Dashboards
if "show_dashboard" in locals():
    if show_dashboard == "C-Level":
        st.header("C-Level Dashboard")
        
        # Top 20 and Bottom 20 Employees by Sales
        employee_sales = sales_data.groupby("Responsible Agent")["SALE PRICE"].sum().reset_index()
        top_20 = employee_sales.nlargest(20, "SALE PRICE")
        bottom_20 = employee_sales.nsmallest(20, "SALE PRICE")
        
        st.subheader("Top 20 Employees by Sales")
        fig_top_20 = px.bar(top_20, x="Responsible Agent", y="SALE PRICE", title="Top 20 Employees")
        st.plotly_chart(fig_top_20, use_container_width=True)
        
        st.subheader("Bottom 20 Employees by Sales")
        fig_bottom_20 = px.bar(bottom_20, x="Responsible Agent", y="SALE PRICE", title="Bottom 20 Employees")
        st.plotly_chart(fig_bottom_20, use_container_width=True)
        
        # Monthly Company Sales Line Chart
        monthly_sales = sales_data.resample("M", on="SALE DATE")["SALE PRICE"].sum()
        st.subheader("Monthly Total Sales")
        fig_monthly_sales = px.line(monthly_sales, x=monthly_sales.index, y="SALE PRICE", title="Monthly Sales")
        st.plotly_chart(fig_monthly_sales, use_container_width=True)

        # Distribution of Residential vs Commercial Units
        st.subheader("Distribution of Residential vs Commercial Units")
        if "residential.units" in inventory_data.columns and "commercial.units" in inventory_data.columns:
            unit_distribution = inventory_data[["residential.units", "commercial.units"]].sum().reset_index()
            unit_distribution.columns = ["Type", "Count"]

            fig_unit_distribution = px.pie(
                unit_distribution,
                names="Type",
                values="Count",
                title="Distribution of Residential and Commercial Units"
            )
            st.plotly_chart(fig_unit_distribution, use_container_width=True)
        else:
            st.write("Residential or Commercial units data is not available.")

        # Sales Heatmap by Region and Month (Top 20 Neighborhoods)
        st.subheader("Sales Heatmap by Region and Month (Top 20 Neighborhoods)")
        if "NEIGHBORHOOD" in sales_data.columns and "SALE DATE" in sales_data.columns:
            # Extract year-month for grouping
            sales_data["Year-Month"] = sales_data["SALE DATE"].dt.to_period("M").astype(str)
    
            # Calculate total sales for each neighborhood
            neighborhood_total_sales = sales_data.groupby("NEIGHBORHOOD")["SALE PRICE"].sum().reset_index()
            # Select top 20 neighborhoods by total sales
            top_20_neighborhoods = neighborhood_total_sales.nlargest(20, "SALE PRICE")["NEIGHBORHOOD"]
    
            # Filter sales data for top 20 neighborhoods
            filtered_sales_data = sales_data[sales_data["NEIGHBORHOOD"].isin(top_20_neighborhoods)]
    
            # Pivot table: calculate total sales for each neighborhood and month
            region_month_sales = filtered_sales_data.pivot_table(
                index="NEIGHBORHOOD", 
                columns="Year-Month", 
                values="SALE PRICE", 
                aggfunc="sum"
            ).fillna(0)
    
            # Create heatmap
            fig_region_month_heatmap = px.imshow(
                region_month_sales,
                labels={"x": "Year-Month", "y": "Neighborhood", "color": "Sales Amount"},
                title="Sales Heatmap by Region and Month (Top 20 Neighborhoods)",
                color_continuous_scale="Viridis"
            )
            st.plotly_chart(fig_region_month_heatmap, use_container_width=True)
        else:
            st.write("NEIGHBORHOOD or SALE DATE data is not available.")

        # 2023 vs 2024 Sales Comparison
        sales_2023 = sales_data[sales_data["SALE DATE"].dt.year == 2023]["SALE PRICE"].sum()
        sales_2024 = sales_data[sales_data["SALE DATE"].dt.year == 2024]["SALE PRICE"].sum()
        
        st.subheader("2023 vs 2024 Sales")
        col1, col2 = st.columns(2)
        col1.metric("2023 Total Sales", f"${sales_2023:,.2f}")
        col2.metric("2024 Total Sales", f"${sales_2024:,.2f}")

    elif show_dashboard == "Agent":
        st.header("Agent Dashboard")

        # Inventory List with Filters
        st.subheader("Filter Inventory")
        st.write("Note: Sales price at 0 means TBD")
        price_tiers = {
            "Below 100k": (0, 100000),
            "100k - 200k": (100000, 200000),
            "200k - 300k": (200000, 300000),
            "300k - 400k": (300000, 400000),
            "Above 400k": (400000, float("inf")),
        }
        selected_tier = st.selectbox("Select Price Tier", list(price_tiers.keys()))
        selected_range = price_tiers[selected_tier]
        
        filtered_inventory = inventory_data[
            (inventory_data["sale.price"] >= selected_range[0]) & (inventory_data["sale.price"] <= selected_range[1])
        ]
        st.dataframe(filtered_inventory)
        
        # Recent 10 Transactions
        filtered_sales_data = sales_data[(sales_data["Responsible Agent"].notnull()) & (sales_data["SALE PRICE"] > 5000)]
        st.subheader("Recent 10 Transactions")
        recent_transactions = filtered_sales_data.nlargest(10, "SALE DATE")[["SALE DATE", "Responsible Agent", "SALE PRICE"]]
        st.dataframe(recent_transactions)
        # Sales Distribution by T10 Agent
        st.subheader("Sales Distribution by Top 10 Agents")
        # Calculate total sales by agent
        agent_sales = filtered_sales_data.groupby("Responsible Agent")["SALE PRICE"].sum().reset_index()
        # Sort by sales in descending order and select the top 10 agents
        top_10_agents = agent_sales.nlargest(10, "SALE PRICE")
        # Create a pie chart for the top 10 agents
        fig_agent_sales = px.pie(
            top_10_agents, 
            names="Responsible Agent", 
            values="SALE PRICE", 
            title="Sales Distribution by Top 10 Agents"
        )
        st.plotly_chart(fig_agent_sales, use_container_width=True)

        # Agent Performance Heatmap by Units Sold
        st.subheader("Agent Performance Heatmap by Units Sold")
        sales_heatmap_data = sales_data.copy()
        # Convert SALE DATE to Year-Month as a string
        sales_heatmap_data["Year-Month"] = sales_heatmap_data["SALE DATE"].dt.to_period("M").astype(str)
        # Pivot table for heatmap (count number of transactions)
        heatmap_df_units = sales_heatmap_data.pivot_table(
            index="Responsible Agent", 
            columns="Year-Month", 
            values="SALE PRICE",  # Any column can be used; we are counting rows
            aggfunc="count"  # Count the number of transactions (units sold)
        ).fillna(0)
        # Create heatmap
        fig_agent_heatmap_units = px.imshow(
            heatmap_df_units, 
            labels={"x": "Year-Month", "y": "Agent", "color": "Units Sold"},
            title="Agent Performance Heatmap by Units Sold",
            color_continuous_scale="Viridis"
        )
        # Display the heatmap
        st.plotly_chart(fig_agent_heatmap_units, use_container_width=True)

        # Weekly Sales Trends
        st.subheader("Weekly Sales Trends")
        # Group sales data by week
        weekly_sales = sales_data.groupby(sales_data["SALE DATE"].dt.to_period("W"))["SALE PRICE"].sum().reset_index()
        weekly_sales.columns = ["Week", "Total Sales"]

        # Convert week periods to string for plotting
        weekly_sales["Week"] = weekly_sales["Week"].astype(str)

        # Create a line chart
        fig_weekly_trends = px.line(
            weekly_sales, 
            x="Week", 
            y="Total Sales", 
            title="Weekly Sales Trends"
        )
        st.plotly_chart(fig_weekly_trends, use_container_width=True)

        # Inventory Status Overview
        st.subheader("Inventory Status Overview")
        inventory_status = inventory_data.copy()
        inventory_status["Price Tier"] = pd.cut(
            inventory_status["sale.price"], 
            bins=[0, 100000, 200000, 300000, 400000, float("inf")], 
            labels=["Below 100k", "100k-200k", "200k-300k", "300k-400k", "Above 400k"]
        )
        price_tier_counts = inventory_status["Price Tier"].value_counts().reset_index()
        price_tier_counts.columns = ["Price Tier", "Count"]

        fig_inventory_status = px.bar(
            price_tier_counts, 
            x="Price Tier", 
            y="Count", 
            title="Inventory Status by Price Tier"
        )
        st.plotly_chart(fig_inventory_status, use_container_width=True)
        
        # Client Appointment Requests
        st.subheader("Client Appointment Requests")
        st.dataframe(pd.DataFrame(st.session_state.appointment_requests))

    elif show_dashboard == "Client":
        st.header("Client Dashboard")
        
        # Submit Appointment Request
        st.subheader("Submit Appointment Request")
        appointment_email = st.text_input("Email")
        appointment_mobile = st.text_input("Mobile")
        appointment_move_in = st.date_input("Expected Move-in Date")
        appointment_time = st.time_input("Time for Visit")
        appointment_area = st.text_input("Expected Area")
        appointment_housing = st.text_input("Desired Housing")
        appointment_agent = st.text_input("Responsible Agent")
        if st.button("Submit Appointment Request"):
            st.session_state.appointment_requests.append({
                "Email": appointment_email,
                "Mobile": appointment_mobile,
                "Move-in Date": appointment_move_in,
                "Time for Visit": appointment_time,
                "Area": appointment_area,
                "Housing": appointment_housing,
                "Agent": appointment_agent,
            })
            st.success("Appointment request submitted!")
        
        # Submit Purchase Request
        st.subheader("Submit Purchase Request")
        purchase_email = st.text_input("Email", key="purchase_email")
        purchase_mobile = st.text_input("Mobile", key="purchase_mobile")
        purchase_move_in = st.date_input("Expected Move-in Date", key="purchase_move_in")
        purchase_payment = st.selectbox("Payment Form", ["Cash", "Card"])
        purchase_address = st.text_input("Address")
        purchase_apartment = st.text_input("Apartment Number")
        purchase_agent = st.text_input("Responsible Agent", key="purchase_agent")
        if st.button("Submit Purchase Request"):
            st.session_state.purchase_requests.append({
                "Email": purchase_email,
                "Mobile": purchase_mobile,
                "Move-in Date": purchase_move_in,
                "Payment": purchase_payment,
                "Address": purchase_address,
                "Apartment Number": purchase_apartment,
                "Agent": purchase_agent,
            })
            st.success("Purchase request submitted!")
        
        # Define price tiers globally
price_tiers = {
    "Below 20k": (0, 20000),
    "20k - 40k": (20000, 40000),
    "40k - 60k": (40000, 60000),
    "60k - 80k": (60000, 80000),
    "Above 80k": (80000, float("inf")),
}

# Client Dashboard: Search Properties
if "show_dashboard" in locals() and show_dashboard == "Client":
    st.header("Client Dashboard")
    
    # Search Inventory
    st.subheader("Search Properties")
    selected_tier = st.selectbox("Select Price Tier", list(price_tiers.keys()))
    selected_range = price_tiers[selected_tier]
    
    # Filter inventory based on selected price tier
    filtered_properties = inventory_data[
        (inventory_data["sale.price"] >= selected_range[0]) & (inventory_data["sale.price"] <= selected_range[1])
    ]
    st.dataframe(filtered_properties)